# Working With Dates And Timestamps To Use With MongoDB

Go over ways to have your MongoDB entries reflect date and time

In [3]:
import bson
import time
import pytz
from datetime import datetime, timedelta

from pymongo import MongoClient

### Initalize A Client, Connect To DB, Speicfy Collection

In [4]:
def get_database(url: str, database_name: str):
    """
    Kết nối đến cơ sở dữ liệu MongoDB, nếu chưa có thì tự động tạo.

    Args:
        url (str): Địa chỉ kết nối MongoDB.
        database_name (str): Tên cơ sở dữ liệu cần kết nối.

    Returns:
        Database | None: Đối tượng cơ sở dữ liệu MongoDB hoặc None nếu có lỗi.
    """
    try:
        client = MongoClient(url)
        db = client[database_name]

        # Kiểm tra nếu database chưa tồn tại thì tạo collection mặc định
        if database_name not in client.list_database_names():
            db.create_collection("default_collection")
            print(f"Database '{database_name}' đã được tạo.")
        else:
            print(f"Kết nối thành công đến database '{database_name}'.")

        return db
    except Exception as e:
        print(f"Lỗi khi kết nối đến MongoDB: {e}")
        return None


if __name__ == "__main__":
    URL = "mongodb+srv://testPythonMongo:o@cluster0.ktuns.mongodb.net/"
    DATABASE_NAME = "my_store"

    database = get_database(URL, DATABASE_NAME)

    if database is not None:  # Sửa kiểm tra điều kiện
        print("Sẵn sàng thao tác với MongoDB!")
        users_col = database["users"]


Kết nối thành công đến database 'my_store'.
Sẵn sàng thao tác với MongoDB!


### Looking at timestamp and datetime objects

In [5]:
time.time()

1740903152.492709

In [6]:
type(time.time())

float

In [7]:
datetime.now()

datetime.datetime(2025, 3, 2, 15, 12, 36, 155649)

In [8]:
datetime.now() + timedelta(days=31)

datetime.datetime(2025, 4, 2, 15, 12, 37, 560422)

### UTC and time zone processing

In [9]:
datetime.utcnow()

datetime.datetime(2025, 3, 2, 8, 12, 39, 438014)

In [10]:
time.timezone

-25200

In [11]:
datetime.fromtimestamp(time.time())

datetime.datetime(2025, 3, 2, 15, 12, 42, 715302)

In [12]:
datetime.fromtimestamp(time.time() + time.timezone)

datetime.datetime(2025, 3, 2, 8, 12, 44, 877579)

### Inserting datetime and timestemp entries

In [14]:
users_col.insert_one(
    {
        "name": "Ethan",
        "join_date_utc": datetime.utcnow(),
        "join_timestamp": time.time() + time.timezone,
        "subscription_end_date_utc": datetime.utcnow() + timedelta(days=31)
    }
)

InsertOneResult(ObjectId('67c412ff5aa9667d1dcbcf26'), acknowledged=True)

### Querying and comparing datetime entries from MongoDB

In [15]:
users_col.find_one({"join_date_utc": {"$gt": datetime(2000, 1, 1)}})

{'_id': ObjectId('67c412fe5aa9667d1dcbcf25'),
 'name': 'Ethan',
 'join_date_utc': datetime.datetime(2025, 3, 2, 8, 12, 46, 333000),
 'join_timestamp': 1740877966.3335164,
 'subscription_end_date_utc': datetime.datetime(2025, 4, 2, 8, 12, 46, 333000)}

In [16]:
find_cursor = users_col.find().sort("join_date_utc", -1)

In [17]:
for document in find_cursor:
    print(document)
    print("\n")

{'_id': ObjectId('67c412ff5aa9667d1dcbcf26'), 'name': 'Ethan', 'join_date_utc': datetime.datetime(2025, 3, 2, 8, 12, 47, 301000), 'join_timestamp': 1740877967.3015056, 'subscription_end_date_utc': datetime.datetime(2025, 4, 2, 8, 12, 47, 301000)}


{'_id': ObjectId('67c412fe5aa9667d1dcbcf25'), 'name': 'Ethan', 'join_date_utc': datetime.datetime(2025, 3, 2, 8, 12, 46, 333000), 'join_timestamp': 1740877966.3335164, 'subscription_end_date_utc': datetime.datetime(2025, 4, 2, 8, 12, 46, 333000)}




### Manually specifying BSON Timestamp

In [18]:
bson.Timestamp(int(time.time()), 1)

Timestamp(1740903174, 1)

In [19]:
users_col.insert_one(
    {
        "name": "Frank",
        "join_date_utc": datetime.utcnow(),
        "join_timestamp": bson.Timestamp(int(time.time() + time.timezone), 1),
        "subscription_end_date_utc": datetime.utcnow() + timedelta(days=31)
    }
)

InsertOneResult(ObjectId('67c413085aa9667d1dcbcf27'), acknowledged=True)

In [20]:
users_col.find_one({"name": "Frank"})

{'_id': ObjectId('67c413085aa9667d1dcbcf27'),
 'name': 'Frank',
 'join_date_utc': datetime.datetime(2025, 3, 2, 8, 12, 56, 44000),
 'join_timestamp': Timestamp(1740877976, 1),
 'subscription_end_date_utc': datetime.datetime(2025, 4, 2, 8, 12, 56, 44000)}

In [21]:
users_col.find_one({"name": "Frank"})["join_timestamp"].time

1740877976

### Converting UTC to local timezone 

In [22]:
find_one_res = users_col.find_one({})
find_one_res["join_date_utc"]

datetime.datetime(2025, 3, 2, 8, 12, 46, 333000)

In [23]:
datetime.fromtimestamp(find_one_res["join_date_utc"].timestamp() - time.timezone)

datetime.datetime(2025, 3, 2, 15, 12, 46, 333000)

In [24]:
pytz.utc.localize(find_one_res["join_date_utc"]).astimezone(pytz.timezone('US/Hawaii'))

datetime.datetime(2025, 3, 1, 22, 12, 46, 333000, tzinfo=<DstTzInfo 'US/Hawaii' HST-1 day, 14:00:00 STD>)

In [25]:
pytz.all_timezones

['Africa/Abidjan',
 'Africa/Accra',
 'Africa/Addis_Ababa',
 'Africa/Algiers',
 'Africa/Asmara',
 'Africa/Asmera',
 'Africa/Bamako',
 'Africa/Bangui',
 'Africa/Banjul',
 'Africa/Bissau',
 'Africa/Blantyre',
 'Africa/Brazzaville',
 'Africa/Bujumbura',
 'Africa/Cairo',
 'Africa/Casablanca',
 'Africa/Ceuta',
 'Africa/Conakry',
 'Africa/Dakar',
 'Africa/Dar_es_Salaam',
 'Africa/Djibouti',
 'Africa/Douala',
 'Africa/El_Aaiun',
 'Africa/Freetown',
 'Africa/Gaborone',
 'Africa/Harare',
 'Africa/Johannesburg',
 'Africa/Juba',
 'Africa/Kampala',
 'Africa/Khartoum',
 'Africa/Kigali',
 'Africa/Kinshasa',
 'Africa/Lagos',
 'Africa/Libreville',
 'Africa/Lome',
 'Africa/Luanda',
 'Africa/Lubumbashi',
 'Africa/Lusaka',
 'Africa/Malabo',
 'Africa/Maputo',
 'Africa/Maseru',
 'Africa/Mbabane',
 'Africa/Mogadishu',
 'Africa/Monrovia',
 'Africa/Nairobi',
 'Africa/Ndjamena',
 'Africa/Niamey',
 'Africa/Nouakchott',
 'Africa/Ouagadougou',
 'Africa/Porto-Novo',
 'Africa/Sao_Tome',
 'Africa/Timbuktu',
 'Africa/